In [4]:
import utils.dataset_functions as df
import utils.user_features as uf
import utils.two_towers as ttn
import torch
import numpy as np
from pathlib import Path
import faiss

# Dimensions of the tower FFN
output_dim      = 16
hidden_dim      = 256
id_dim          = 16


# Training
num_epochs      = 50
learning_rate   = 1e-3
batch_size      = 256
patience        = 5
device='cuda' if torch.cuda.is_available() else 'cpu'
print("Training models on:", device)

Training models on: cuda


### Recommendations & Evaluation
Ik weet niet precies hoe alles precies in elkaar steekt dus ik schrijf ff een skelet en dan moet je maar ff kijken of dingen kloppen.

Dit zijn de stappen die ik probeer te zetten:
- Lijst van nummers inladen
- Lijst van users inladen
- modellen inladen
- per model
  - bereken song embeddings (yv)
  - bereken user embeddings (uv)
  - maak index voor reccomendations
  - per user
    - maak recommendations
    - voeg recommendations en test set (van die user) samen
    - reken dot-product uit per query recommendation pair
    - rerank op dot-product
    - beoordeel rankings (op reinforcement signal / like-dislikes


    

In [ ]:
songs = None # Lijst van nummers inladen (los van users) (features + IDs)
users = None # Lijst van users inladen (los van users) (features + IDs)


models = ["binary_label", "continueous_label"]



for label_id, model_name in enumerate(models):
    test_set = df.load_tensor_dataloader("val", Path("dataset")/"processed", batch_size, label_id)

    # Load model
    model = ttn.DualAugmentedTwoTower(model_name, hidden_dim, output_dim, id_dim)
    model.load(Path("models")/ f"{model_name}.pt")
    
    # Create song embeddings 
    # (Ik weet ff niet hoe de columns heten, dubbelcheck dit ff)
    songs['ys'] = songs['song_features', 'song_id'].apply(lambda song: model.song_pass(song['song_features'], song['song_id']), axis = 1)

    # Create user embeddings
    # (Ik weet ff niet hoe de columns heten, dubbelcheck dit ff)
    users['yu'] = users['user_features', 'user_id'].apply(lambda user: model.user_pass(user['user_features'], user['user_id']), axis = 1)
    
    # Create index for reccommendations
    model.create_index(songs['ys'].to_numpy())

    # Metric voor alle recommendations
    RS_scores = []  # Reinforcement Signal
    LDL_scores = [] # Like-DisLike

    # Loop users
    for index, user in users.iterrows:

        # Get user embedding
        query = user['yu']

        # Make 25 recommendations
        indices, distances = model.recommendations(query, 25)
        recommendations = songs.iloc[indices]

        # Merge recommendations with the users test-split
        
        # recommendations = ... (add test songs here)

        # Calculate scores (dot product with query)
        recommendations['score'] = recommendations['ys'].apply(lambda yv: (query * yv).sum())

        # Re-rank
        ranking = recommendations.sort_values(by = 'score')

        # Evaluate ranking
        # Weet nog niet hoe. We moeten iig de like/dislikes mergen met de ranking en daar iets van een score uit halen
        # Ook kunnen we nog de reinforcement signal mergen en daar Froukjes metrics op los laten
        
        # RS_score = ...
        # LDL_score = ...

        # RS_scores.append(RS_score)
        # LDL_scores.append(LDL_score)

    # Aggregate scores (average?)
    RS_score = np.mean(RS_scores)
    LDL_score = np.mean(LDL_scores)
    print(f'{model_name}: RS = {RS_score}, LDL = {LDL_score}')